In [21]:
# Python 3.8 version
import rawpy
import glob
import os
import numpy as np
import tifffile as tif
import time
from tqdm import tqdm

# Initial setting

In [22]:
# Observation scene
SceneID = "Kotogahama-20231228-1153-P1"

# I/O Path
Dir1   = "I_InputFiles/"
Dir2   = "I_Raw/"
Dir3   = "O_Tiff/"

# Input files
IFile1 = Dir1 + SceneID+ "_ListImages.csv"         # List of raw images
IFile2 = Dir1 + "P1-FlatCofficient.npz"            # Lens Shading Cofficient

# All original image processing
PhotoIDs = []
with open(IFile1, "r", encoding="utf-8") as f:
    Photolist = f.readlines()
for i in Photolist:
    PhotoIDs.append(i.rstrip())

# Input lens shading correction coefficients
FlatCo_npz     = np.load(IFile2)
FlatCofficient = FlatCo_npz["arr_0"]

# Class to save in Tiff format

In [23]:
class ImageParameterEach:
    def __init__(self, photo):
        self.raw_original = rawpy.imread(Dir2+photo+".DNG")
        self.OFile1       = Dir3 + photo + ".tif"
        # Debayering
        self.RGR_original = self.raw_original.postprocess(
                        demosaic_algorithm = rawpy.DemosaicAlgorithm.LINEAR,
                        output_color       = rawpy.ColorSpace.raw,
                        user_wb            = [2.546875, 1.0, 1.83203125, 0.0],
                        output_bps         = 16,
                        no_auto_scale      = False,
                        gamma              = [1,0]
                        )
        # Lens Shading
        self.CorrectedImage1 = np.round(self.RGR_original / FlatCofficient)
        self.CorrectedImage2 = np.where(self.CorrectedImage1 > 65535, 65535, self.CorrectedImage1).astype(np.uint16)

# Debayer, Correction and Save

In [24]:
for i in tqdm(PhotoIDs):
    photo = ImageParameterEach(i)
    tif.imwrite(photo.OFile1, photo.CorrectedImage2.astype('uint16'))

100%|██████████| 1/1 [00:04<00:00,  4.46s/it]
